# MNIST classifier tutorial

* MNIST database of handwritten digits is a popular dataset to demonstrate machine learning classifier training.
* In this tutorial, we train a basic Neural Network (NN) classifier using PyTorch.
* Once the basic NN classifier training workflow has been defined, we parallelize the mutually independent tasks using Covalent.
* This tutorial has two primary objectives.
* One goal is to show the ease with which a "normal" workflow can be adapted to a Covalent workflow.
* The second objective is to display the browser-based Covalent workflow tracking UI.

In [1]:
# !pip install cova
# !conda install pytorch torchvision -c pytorch -y

In [14]:
import covalent as ct

import torch
import torch.nn.functional as F

from pathlib import Path

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from typing import Tuple

In [3]:
# # Add global variables inside the workflow 
# n_epochs = 2
# batch_size_train = 64
# batch_size_test = 1000
# learning_rate = 0.01
# momentum = 0.5
# log_interval = 10

# random_seed = 1
# torch.backends.cudnn.enabled = False
# torch.manual_seed(random_seed)
# torch.autograd.set_detect_anomaly(True)

# network = Net()

In [15]:
@ct.electron
def data_loader(batch_size: int,
                train: bool,
                download: bool = True,
                shuffle: bool = True,
                data_dir: str = '~/data/mnist/',
                ) -> torch.utils.data.dataloader.DataLoader:
    """MNIST data loader."""

    data_dir = Path(data_dir).expanduser()
    data_dir.mkdir(parents=True, exist_ok=True)

    data = datasets.MNIST(
        data_dir,
        train=train,
        download=download,
        transform=ToTensor()
        )

    return DataLoader(data, batch_size=batch_size, shuffle=shuffle)

In [17]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [18]:
@ct.electron
def get_optimizer(model: NeuralNetwork, learning_rate: float, momentum: float) -> \
        torch.optim.Optimizer:
    """Get Stochastic Gradient Descent optimizer."""

    return torch.optim.SGD(model.parameters(), learning_rate, momentum)

In [19]:
@ct.electron
def train_over_one_epoch(dataloader: torch.utils.data.dataloader.DataLoader,
                         model: NeuralNetwork,
                         optimizer: torch.optim.Optimizer,
                         log_interval: int,
                         epoch: int,
                         loss_fn,
                         train_losses: list,
                         train_counter: int,
                         device: str = "cpu"):
    """Train neural network model over one epoch."""

    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % log_interval == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

            train_losses.append(loss)
            train_counter.append(
                (batch * 64) + ((epoch - 1) * len(dataloader.dataset)))

    return model, optimizer

In [20]:
@ct.electron
def test(dataloader: torch.utils.data.dataloader.DataLoader,
         model: NeuralNetwork,
         loss_fn: callable,
         device: str = "cpu") -> None:
    """Test the model performance."""

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [21]:
@ct.electron
def train_model(train_dataloader: torch.utils.data.dataloader.DataLoader,
                train_losses: list,
                train_counter: int,
                log_interval: int,
                model: NeuralNetwork,
                optimizer: torch.optim.Optimizer,
                loss_fn: callable,
                epochs: int,
                results_dir: str = '~/data/mnist/results/') -> Tuple[NeuralNetwork, ]:
    """Train neural network model."""

    results_dir = Path(results_dir).expanduser()
    results_dir.mkdir(parents=True, exist_ok=True)

    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}\n-------------------------------")
        model, optimizer = train_over_one_epoch(dataloader=train_dataloader,
                                                model=model,
                                                optimizer=optimizer,
                                                train_losses=train_losses,
                                                train_counter=train_counter,
                                                log_interval=log_interval,
                                                epoch=epoch,
                                                loss_fn=loss_fn)

    # Save model and optimizer
    torch.save(model.state_dict(), f'{results_dir}model.pth')
    torch.save(optimizer.state_dict(), f'{results_dir}optimizer.pth')
    return model, optimizer

In [22]:
@ct.lattice
def workflow(model: NeuralNetwork,
             epochs: int = 2,
             batch_size_train: int = 64,
             batch_size_test: int = 1000,
             learning_rate: float = 0.01,
             momentum: float = 0.5,
             log_interval: int = 10,
             loss_fn: callable = F.nll_loss):
    """MNIST classifier training workflow"""

    train_dataloader = data_loader(batch_size=batch_size_train, train=True)
    test_dataloader = data_loader(batch_size=batch_size_test, train=False)

    train_losses, train_counter, test_losses = [], [], []
    optimizer = get_optimizer(model=model, learning_rate=learning_rate, momentum=momentum)
    model, optimizer = train_model(
        train_dataloader=train_dataloader,
        train_losses=train_losses,
        train_counter=train_counter,
        log_interval=log_interval,
        model=model,
        optimizer=optimizer,
        loss_fn=loss_fn,
        epochs=epochs)
    test(dataloader=test_dataloader, model=model, loss_fn=loss_fn)

In [23]:
    import time

    start = time.time()
    workflow(model=NeuralNetwork().to("cpu"),)
    end = time.time()
    print(f"Regular workflow takes {end - start} seconds.")

Epoch 1
-------------------------------
loss: 2.320409  [    0/60000]


/var/folders/hn/dnjxkq7946j9wpxv1cy9pm2m0000gn/T/ipykernel_70432/4014429098.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


loss: 2.307707  [  640/60000]
loss: 2.311337  [ 1280/60000]
loss: 2.311996  [ 1920/60000]
loss: 2.303629  [ 2560/60000]
loss: 2.306665  [ 3200/60000]
loss: 2.290847  [ 3840/60000]
loss: 2.314099  [ 4480/60000]
loss: 2.291214  [ 5120/60000]
loss: 2.301382  [ 5760/60000]
loss: 2.289404  [ 6400/60000]
loss: 2.291337  [ 7040/60000]
loss: 2.302155  [ 7680/60000]
loss: 2.301734  [ 8320/60000]
loss: 2.291988  [ 8960/60000]
loss: 2.279692  [ 9600/60000]
loss: 2.286411  [10240/60000]
loss: 2.284787  [10880/60000]
loss: 2.287410  [11520/60000]
loss: 2.276541  [12160/60000]
loss: 2.269667  [12800/60000]
loss: 2.275173  [13440/60000]
loss: 2.300417  [14080/60000]
loss: 2.263923  [14720/60000]
loss: 2.266652  [15360/60000]
loss: 2.251887  [16000/60000]
loss: 2.253369  [16640/60000]
loss: 2.259032  [17280/60000]
loss: 2.251317  [17920/60000]
loss: 2.242465  [18560/60000]
loss: 2.242042  [19200/60000]
loss: 2.176988  [19840/60000]
loss: 2.204401  [20480/60000]
loss: 2.224599  [21120/60000]
loss: 2.21

In [13]:
start = time.time()
dispatch_id = ct.dispatch(workflow)()
print(dispatch_id)
result = ct.get_result(dispatch_id=dispatch_id, wait=True)
print(result.result)
end = time.time()
print(f"Covalent workflow execution time: {end - start}")

3fb152ec-da1c-4149-a3c0-4bab582b44ae
None
Covalent workflow execution time: 37.76052188873291


In [24]:
start = time.time()
dispatch_id = ct.dispatch(workflow)(model=NeuralNetwork().to("cpu"),)
print(dispatch_id)
result = ct.get_result(dispatch_id=dispatch_id, wait=True)
print(result.result)
end = time.time()
print(f"Covalent workflow takes {end - start} seconds.")

bae16d96-3deb-4894-b016-b1a2500d4a29
None
Covalent workflow takes 45.22564506530762 seconds.


Triple check to ensure there is no plagiarism issues
References:
- https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html